## Grid reconstruction

Each image may have small rotations or areas that overlap with other input images that need to be considered when reconstructing the grid.


To reconstruct the grid, I try and get the biggest contour (grid pattern) on a processed image. Then I resize the contour and rotate the grid to get the output image. Finally, I put them together. Only the central part of the Neubauer chamber will be considered.

In [46]:
import numpy as np
import cv2

In [47]:
DEBUG_RECONSTRUCTION = True

Read image (note that any image of the dataset could be chosen. We will use 1Quad.jpg as an example)

In [48]:
img = cv2.imread("./dataset/1Quad.jpg")

Grid reconstruction - Copy image, grayscale, and adaptive threshold

In [49]:
image = img.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 57, 5)
if (DEBUG_RECONSTRUCTION): cv2.imwrite("thresh.jpg", thresh)

Generated Threshold image:
![Threshold image](./thresh.jpg)

Mathematical morphology - apply erosion, morph_close and erosion again to keep only (kinda) the grid

In [50]:
thresh = cv2.GaussianBlur(thresh, (7,7), 0)
ret, thresh = cv2.threshold(thresh, 210, 255, 0)
kernel = np.ones((3,3), np.uint8)
morphed = cv2.erode(thresh, None, kernel, iterations=16)
morphed = cv2.morphologyEx(morphed, cv2.MORPH_CLOSE, kernel, iterations=55)
morphed = cv2.erode(morphed, None, kernel, iterations=10)
if (DEBUG_RECONSTRUCTION): cv2.imwrite("morphed.jpg", morphed)

Generated Morphed image:
![Morphed image](./morphed.jpg)

Then, we find the contours and get the biggest one (that should be the inner part of the Neubauer chamber)

In [51]:
contours, hierarchy = cv2.findContours(morphed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

c = max(contours, key = cv2.contourArea)

We have to rescale the contour due to the previously morph operations

In [52]:
def scale_contour(cnt, scale):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

# Resize contour to match real image
c = scale_contour(c, 1.16)

But the image is still rotated. For this reason we need to get the corners of contour (otherwise we would not know how much rotation we need). Finally we crop them to the correct size.

In [53]:
rect = cv2.minAreaRect(c)
box = cv2.boxPoints(rect)
box = np.int0(box)

def crop_rect(img, rect):
    center = rect[0]
    size = rect[1]
    angle = rect[2]
    center, size = tuple(map(int, center)), tuple(map(int, size))

    height, width = img.shape[0], img.shape[1]
    if (DEBUG_RECONSTRUCTION): print("width: {}, height: {}".format(width, height))

    M = cv2.getRotationMatrix2D(center, angle, 1)
    img_rot = cv2.warpAffine(img, M, (width, height))

    img_crop = cv2.getRectSubPix(img_rot, size, center)

    img_crop = cv2.rotate(img_crop, rotateCode=cv2.ROTATE_90_CLOCKWISE)

    return img_crop, img_rot

# Rotate and crop images
img_crop, img_rot = crop_rect(img, rect)

if (DEBUG_RECONSTRUCTION): cv2.imwrite("croped.jpg", img_crop)

width: 2000, height: 1431


Generated Rotated and croped image:

![Rotated and croped image](./croped.jpg)

The last step is to put all images together. For this we have a function that concat images vertically and horizontally.

In [54]:
def vertical_concat(im_list, interpolation=cv2.INTER_CUBIC):
    w_min = min(im.shape[1] for im in im_list)
    im_list_resize = [cv2.resize(im, (w_min, int(im.shape[0] * w_min / im.shape[1])), interpolation=interpolation) for im in im_list]
    return cv2.vconcat(im_list_resize)

def horizontal_concat(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation) for im in im_list]
    return cv2.hconcat(im_list_resize)

def concat(im_list_2d, interpolation=cv2.INTER_CUBIC):
    im_list_v = [horizontal_concat(im_list_h, interpolation=cv2.INTER_CUBIC) for im_list_h in im_list_2d]
    return vertical_concat(im_list_v, interpolation=cv2.INTER_CUBIC)

reconstructed = concat([[img_crop, img_crop, img_crop, img_crop, img_crop],
                        [img_crop, img_crop, img_crop, img_crop, img_crop],
                        [img_crop, img_crop, img_crop, img_crop, img_crop],
                        [img_crop, img_crop, img_crop, img_crop, img_crop],
                        [img_crop, img_crop, img_crop, img_crop, img_crop]])

if (DEBUG_RECONSTRUCTION): cv2.imwrite("reconstructed.jpg", reconstructed)

We end up with the following reconstructed grid:
![Reconstructed grid](./reconstructed.jpg)